In [1]:
from scipy import optimize
from bingo.evaluation.fitness_function import VectorBasedFunction
import numpy as np
import sys
from numpy import array

In [3]:
number_of_eqps_values = 7
eqps_values_np = np.array([ i*0.002 for i in range(number_of_eqps_values) ])
P_desired = np.array([[1, -0.5, -0.5],
                    [-0.5, 1, -0.5],
                    [-0.5, -0.5, 1]])
dataloc = 'processed_data/vm_1_bingo_format.txt'
pre_data = np.loadtxt(dataloc)
training_data = []
eqpses = []
for row in pre_data:
    if np.isnan(row[0]) == False:
        training_data.append(row[:3])
        eqpses.append(row[3])
training_data = np.array(training_data)
eqpses = np.array(eqpses)
#print(training_data)

def mapping_(x):

    # mapping = lambda X_0: array([[x[0], x[1], x[2]],
    #    [x[3], x[4], x[5]],
    #    [x[6], x[7], x[8]]]) + (X_0) * (array([[x[9], x[10], x[11]],
    #    [x[12], x[12], x[13]],
    #    [x[14], x[15], x[16]]]))
    mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], x[3], x[4] ],  
        [x[3] , x[6], x[5]],
        [x[4], x[5], x[7]]]))
    mapping = lambda X_0: (x[1]*X_0 + x[0])  * (array([[x[2], 0, 0 ],  
        [0 , x[6], 0],
        [0, 0, x[7]]]))
    
    mapping_matrices = np.array([mapping(e) for e in eqpses])
    P_mapped = mapping_matrices.transpose((0, 2, 1)) @ P_desired @ mapping_matrices
    back_calculated_yield_stresses = []
    for i,stresses in enumerate(training_data):
        #print('here we go', np.transpose(training_data[i]), P_mapped[i], training_data[i])
        back_calculated_yield_stresses.append(P_mapped[i] @ training_data[i])

    error = training_data - back_calculated_yield_stresses
    return error.flatten()



# ((array(0.00500253) + array(0.00500253))/(X_0 + array(0.00500253) + array(0.00500253)) ) * (array([[-0.87240378, -1.01069406, -0.3899734 ],  
#        [-0.8733965 , -1.02045703, -0.37967495],
#        [-0.86253049, -1.02066836, -0.38872487]]))


In [4]:
x0 = [-0.83111601, -0.46657978, -0.06323608, -0.83507571, -0.45586107, -0.0702321, -0.84231627, -0.46026723, -0.05982535, 
      67.83825554, 11.92019618, 22.12241255, 67.75485734, 11.34285823, 22.7518725, 68.42081951, 11.30870116, 22.22108227
]
x0 = [ 0.00500253*2,0.00500253*2,-0.87240378, -1.01069406, -0.3899734, -0.8733965 , -1.02045703, -0.37967495 ]
mapping_(x0)
#results = optimize.minimize(mapping_, )

array([   1.72512007,   -6.43866597,    4.71388092, ...,   35.87938357,
       -133.91656864,   98.04721948])

In [6]:
results = optimize.root(mapping_, x0=x0, method='lm', options={"ftol": 1e-16, "xtol": 1e-16, "gtol": 1e-16, "maxiter": 100000})
results

   cov_x: None
    fjac: array([[ 5.28136170e+04, -4.07311408e+02, -3.95720638e+02, ...,
         9.95445605e-03, -3.71505364e-02,  2.71960801e-02],
       [-4.07721557e+02,  4.49971331e+02, -2.22991338e+02, ...,
        -7.56826458e-03,  4.92825468e-02,  8.06074162e-03],
       [-3.96119116e+02, -2.24300554e+02,  3.88584156e+02, ...,
        -1.99554466e-02, -4.29340653e-03,  3.75463852e-02],
       ...,
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])
     fun: array([-1.20499917e-08,  4.49711832e-08, -3.29211902e-08, ...,
        8.10310823e-08, -3.02412104e-07,  2.21381043e-07])
    ipvt: array([1, 7, 3, 8, 5, 6, 2, 4], dtype=int32)
 message: 'The relative error b

In [9]:
np.set_printoptions(threshold=sys.maxsize)
mapping_(results.x)

array([-1.20499917e-08,  4.49711832e-08, -3.29211902e-08, -2.23928844e-08,
        8.35713863e-08, -6.11785005e-08, -3.11571506e-08,  1.16280077e-07,
       -8.51229220e-08, -3.83427921e-08,  1.43097260e-07, -1.04754459e-07,
       -4.39498073e-08,  1.64022925e-07, -1.20073111e-07, -4.79781956e-08,
        1.79057075e-07, -1.31078870e-07, -5.04279587e-08,  1.88199721e-07,
       -1.37771750e-07, -5.12990930e-08,  1.91450837e-07, -1.40151734e-07,
       -5.05916038e-08,  1.88810453e-07, -1.38218837e-07, -4.83054876e-08,
        1.80278569e-07, -1.31973060e-07, -4.44407462e-08,  1.65855170e-07,
       -1.21414395e-07, -3.89973778e-08,  1.45540227e-07, -1.06542835e-07,
       -3.19753894e-08,  1.19333805e-07, -8.73583872e-08, -2.33747670e-08,
        8.72358470e-08, -6.38610516e-08, -1.31955353e-08,  4.92464238e-08,
       -3.60508636e-08, -1.43765533e-09,  5.36542188e-09, -3.92775235e-09,
        1.18988446e-08, -4.44070452e-08,  3.25082397e-08,  2.68139679e-08,
       -1.00071063e-07,  

In [13]:
np.set_printoptions(precision=32)
results.x

array([ 1.3653212555775154e-01, -3.1543795562334303e-09,
       -5.9802524896121678e+00, -1.0106940600000001e+00,
       -3.8997340000000003e-01, -8.7339650000000002e-01,
       -5.9802524896121678e+00, -5.9802524896121678e+00])

In [ ]:
mapping = lambda X_0: array([[-0.83111601, -0.46657978, -0.06323608],
       [-0.83507571, -0.45586107, -0.07023218],
       [-0.84231627, -0.46026723, -0.05982535]]) + (X_0) * (array([[67.83825554, 11.92019618, 22.12241255],
       [67.75485734, 11.34285823, 22.75187253],
       [68.42081951, 11.30870116, 22.22108227]]))
Q = mapping(0)
Q